In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from ipmcmc.generate_data import *
from ipmcmc.linear_gaussian_state_model import *
from ipmcmc.non_linear_gaussian_state_model import *
from ipmcmc.smc import *
from ipmcmc.csmc import *
from ipmcmc.ipmcmc import *
from ipmcmc.estimation import *    


We implemented the same models as those in the paper. Each one of them is implemented and generates observations and states in the two next cells

In [2]:
# 4.1. Linear Gaussian State Space Model
np.random.seed(420)
# Parameters
t_max = 50
n_particles = 100

r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)



l_transition_model = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearTransition(
    default_mean=np.zeros(3), default_cov=omega, default_alpha=alpha) for t in range(1, t_max)]
l_proposals = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearProposal(
    default_mean=np.zeros(3), default_cov=omega, default_alpha=alpha) for t in range(1, t_max)]
l_observation_model = [LinearObservation(default_mean=np.zeros(
    20), default_cov=sigma, default_beta=beta) for t in range(0, t_max)]

# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(
    t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma,
    transition_coeffs=alpha, observation_coeffs=beta)

In [3]:
# 4.2. Nonlinear State Space Model
np.random.seed(420)
nl_mu = 0
start_std = np.sqrt(5)
omega = np.sqrt(10)
sigma = np.sqrt(10)

nl_transition_model = [NonLinearMu(default_mean=nl_mu, default_std=start_std)]+[NonLinearTransition(
    default_mean=0, default_std=omega) for t in range(1, t_max)]
nl_proposals = [NonLinearMu(default_mean=nl_mu, default_std=start_std)]+[
    NonLinearProposal(default_mean=0, default_std=omega) for t in range(1, t_max)]
nl_observation_model = [NonLinearObservation(
    default_mean=0, default_std=sigma) for t in range(0, t_max)]

nl_states, nl_observations = nonlinear_gaussian_state_space(
    t_max=t_max, mu=nl_mu, start_std=start_std, transition_std=omega, noise_std=sigma)

In [4]:
# ipmcmc run: works with both linear and non-linear models.
# It is pretty long to run, longer for the linear model which has 3-dimensional states.
# For the linear model, each MCMC step take approximately 90 secs, and 80 secs for 
# the non-linear, on our computers.

n_nodes = 32
n_conditional_nodes = 16
n_steps = 5

linear= True

if linear:
    print('init_conditional_traj')
    
    init_conditional_traj = np.zeros((n_conditional_nodes, t_max, len(mu)))
    for i in tqdm(range(n_conditional_nodes)):
        particles, _, _ = smc(l_observations, n_particles,
                              l_transition_model, l_proposals, l_observation_model)
        init_conditional_traj[i] = particles.mean(axis=1)

    print('running ipmcmc')
    particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
        n_steps, n_nodes, n_conditional_nodes, l_observations, n_particles, init_conditional_traj,
        l_proposals, l_transition_model, l_observation_model)

else:
    print('init_conditional_traj')
    
    init_conditional_traj = np.zeros((n_conditional_nodes, t_max, 1))
    for i in tqdm(range(n_conditional_nodes)):
        particles, _, _ = smc(nl_observations, n_particles,
                              nl_transition_model, nl_proposals, nl_observation_model)
        init_conditional_traj[i] = particles.mean(axis=1)

    print('running ipmcmc')
    particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
        n_steps, n_nodes, n_conditional_nodes, nl_observations, n_particles, init_conditional_traj,
        nl_proposals, nl_transition_model, nl_observation_model)
        

  0%|          | 0/16 [00:00<?, ?it/s]

init_conditional_traj


  0%|          | 0/5 [00:00<?, ?it/s]

running ipmcmc


100%|██████████| 5/5 [08:52<00:00, 106.41s/it]


In [ ]:
# Mean estimation for the linear model, using kalman filter and rts smoother as ground truth
# Make sure that the particles used are the one generated during a run of the ipmcmc sampler
# for the linear model

true_means, true_covs = compute_ground_truth(l_observations, mu, start_var, alpha, omega, beta, sigma)

rao_black_traj = rao_blackwellisation(particles, weights, zetas, n_conditional_nodes)

errors_function_of_mcmc_step = []
errors_function_of_state_step = []
for r in range(1, (n_steps+1)):
    errors_function_of_mcmc_step.append(compute_error(rao_black_traj, true_means, r))

for t in range(1, (t_max+1)):
    errors_function_of_state_step.append(compute_error(rao_black_traj, true_means, state_step=t))